In [ ]:
#import requests
import time
import re
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from extract.common import config
from extract.common import credentials
from extract.common import browser

In [ ]:
class LogInPage(object):
    def __init__(self, investment_site_uid):
        self._config = config()['investment_sites'][investment_site_uid]
        self._credentials = credentials()['investment_sites'][investment_site_uid]
        self._browser = browser()
        self._home = "{}{}".format(self._config['url'], self._config['login']['url'])

    
    def type_user_credential(self):
        input_ = self._browser.find_element_by_xpath(self._config['login']['user'])
        input_.send_keys(self._credentials['user'])
        
    def type_password_credential(self):
        input_ = self._browser.find_element_by_xpath(self._config['login']['password'])
        input_.send_keys(self._credentials['password'])
        
    def click_login(self):
        self._browser.find_element_by_xpath(self._config['login']['button']).click()
    
    def _auth0(self):
        if 'auth0' in self._config['login']:
            element = WebDriverWait(self._browser, 10).until(
                EC.presence_of_element_located((By.XPATH, self._config['login']['auth0']))
            )
            self._browser.find_element_by_xpath(self._config['login']['auth0']).click()
        
    def navigate(self):
        self._browser.get(self._home)
        self._browser.implicitly_wait(10)
        self._auth0()
        self.type_user_credential()
        self.type_password_credential()
        self.click_login()
        
class ProjectListPage(object):

    def __init__(self, investment_site_uid):
        self._config = config()['investment_sites'][investment_site_uid]
        self._browser = browser()
        self._soup = None
    @property
    def project_urls(self):
        pass
    
    def navigate(self):
        self._browser.get(self._home)
        if 'wait' in self._config:
            WebDriverWait(self._browser, 10).until(
                EC.element_to_be_clickable(
                    (By.XPATH, self._config['wait'])
                )
            )
        self._browser.implicitly_wait(50)
        html = self._browser.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
        self._browser.implicitly_wait(50)
        self._soup=BeautifulSoup(self._browser.page_source, 'html.parser')
        
class BriqProjectListPage(ProjectListPage):
    
    def __init__(self):
        ProjectListPage.__init__(self,'briq')
        self._home = "{}{}".format(self._config['url'],self._config['home'])
    @property
    def project_urls(self):
        urls = [i.parent['href'] 
                        for i in self._soup.find_all('img', {'src': re.compile(r'label_buscando-inversion')})]
        return urls

class CumploProjectListPage(ProjectListPage):
    def __init__(self):
        ProjectListPage.__init__(self,'cumplo')
        self._home = "{}{}".format(self._config['url'],self._config['home'])
        
    @property
    def project_urls(self):       
        project_urls = [i.find_parents('div', limit=1)[0].text[3:] for i in self._soup.find_all(text='ID')]
        project_urls = ['{}{}'.format('/solicitud/MX/',i) for i in project_urls]

        return project_urls

In [ ]:

login_page = LogInPage('cumplo')
login_page.navigate()
project_page = CumploProjectListPage()
project_page.navigate()
print(project_page.project_urls)
